In [2]:
!pip install sentence_transformers


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# imports
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load your data
df = pd.read_csv('argumentdetection8.csv')

print(f"Total sentences: {len(df)}")
print(df['category'].value_counts())


Total sentences: 897408
category
evidence    833983
claim        38749
non_info     24676
Name: count, dtype: int64


In [5]:
# Take a stratified sample 
sample_size = 75000

if len(df) > sample_size:  # take either all rows in that category OR sample_size/3, whichever is smaller
    df_sample = df.groupby('category', group_keys=False).apply(
        lambda x: x.sample(min(len(x), sample_size // 3), random_state=11)
    )
else:
    df_sample = df

print(df_sample['category'].value_counts())


category
claim       25000
evidence    25000
non_info    24676
Name: count, dtype: int64


/tmp/ipykernel_2299/2770587651.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = df.groupby('category', group_keys=False).apply(


In [6]:
# Load sentence transformer
model = SentenceTransformer('all-MiniLM-L6-v2')
# Generate embeddings
embeddings = model.encode(df_sample['sentence'].tolist(), 
                         show_progress_bar=True,
                         batch_size=32)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, 
    df_sample['category'], 
    test_size=0.2, 
    random_state=42,
    stratify=df_sample['category']
)

# Train simple classifier
classifier = LogisticRegression(max_iter=1000, random_state=11)
classifier.fit(X_train, y_train)

# Evaluate
y_pred = classifier.predict(X_test)

print("Classification Report")
print(classification_report(y_test, y_pred))

print("Confusion Matrix")
cm = confusion_matrix(y_test, y_pred, labels=['claim', 'evidence', 'non_info'])
print(cm)

Batches: 100%|██████████| 2334/2334 [12:26<00:00,  3.13it/s]


Classification Report
              precision    recall  f1-score   support

       claim       0.96      0.96      0.96      5000
    evidence       0.88      0.87      0.88      5000
    non_info       0.87      0.88      0.88      4936

    accuracy                           0.90     14936
   macro avg       0.90      0.90      0.90     14936
weighted avg       0.90      0.90      0.90     14936

Confusion Matrix
[[4785  106  109]
 [ 116 4367  517]
 [  98  506 4332]]
